In [1]:
import skimage.io as io
import pandas as pd
import os
import logging
example_root = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 'examples')

___Set up Logger___

In [2]:
logger = logging.getLogger('pcna')
logger.setLevel(logging.DEBUG)
handler1 = logging.StreamHandler()
handler1.setLevel(logging.INFO)
logger.addHandler(handler1)

## Hack In Intermediate Steps in pcnaDeep

### Author: Yifan Gui @ Kuan Yoow Chan group

In this tutorial we discuss:
1. Scripts to perform instance segmentation only. You can correct segmentation errors through VIA2 software.
2. Approaches to perform downstream analysis from intermediate ground truth files.


### 1 Perform Instance Segmentation only

The script `bin/detect.py` is dedicated for such purpose. It can output VIA2-readable `json` file of the instance segmentation result.

Use the following command to run on demo data. This will generate a file named `MCF10A_demo.json` (37MB) under `examples/intermediate` folder.

`python detect.py --pcna ../examples/demo/MCF10A_demo_PCNA.tif --bf ../examples/demo/MCF10A_demo_BF.tif --config-file ../config/dtrnCfg.yaml --output ../examples/intermediate/ --prefix MCF10A_demo --sat 1 --gamma 1 --opts MODEL.WEIGHTS ../models/mrcnn_sat_rot_aug.pth`

#### Command line parameters

- __`--bf`__, __`--pcna`__ Path to the bright field and PCNA fluorescent image files.
- __`--sat`__, __`--gamma`__ The pre-processing parameters, i.e., pixel saturation and gamma correction factor. Default 1 for both.
- __`--prefix`__ The prefix of image name in the `json` output.
    - The image file name will be like `prefix-0001.png` in the `json`. You should save image files for labeling in the same way, so that VIA2 knows the image identity. You may find `pcnaDeep.data.utils.save_seq` function useful, as explained in the [Mask R-CNN training tutorial](train_pcna_detectron2.ipynb) (also check out this tutorial for importing `json` into the VIA2 software).
- __`--config-file`__ Path to Detectron2 config file. Default `dtrnCfg.yaml`.
- __`--confidence-threshold`__, __`--opts`__ Same as the main application, see [Getting Started](getting_started.ipynb).

---

### 2 Begin from detection (instance segmentation) ground truth

The detection ground truth is embodied as `json` file. To track and resolved from this ground truth, we can use `pcnaDeep.tracker.track_GT_json`. 

In [3]:
from pcnaDeep.tracker import track_GT_json
from pcnaDeep.refiner import Refiner
from pcnaDeep.resolver import Resolver
help(track_GT_json)

Help on function track_GT_json in module pcnaDeep.tracker:

track_GT_json(fp_json, height=1200, width=1200, displace=40, gap_fill=5, size_min=100, fp_intensity_image=None, fp_pcna=None, fp_bf=None, sat=None, gamma=None)
    Track ground truth VIA json file. Wrapper of `track_mask()`
    
    Args:
        fp_json (str): file path to the json file.
        height (int): pixel height of the mask corresponding to GT json.
        width (int): pixel width of the mask corresponding to GT json.
        displace (int): distance restriction, see `track()`.
        gap_fill (int): time restriction, see `track()`.
        size_min (int): remove object smaller then some size, in case the mask labeling is not precise.
        fp_intensity_image (str): optional image file path, if supplied, will extract fore/backgound PCNA intensity, and
            bright field intensity/std for tracking.
            Must has the same shape as mask, so will override height and width.
        fp_pcna (str): optiona

_Note:_ Because we are working with the ground truth, we set all thresholds related to the Greedy Phase Searching to 1.

The following code will generate a __`tracked object table`__ named `refined.csv` for following analysis. When making ground truth, it is practical to check `phase.csv` to see if there are any faulty phases (usually indicates wrong tracking).

You can correct the `refined.csv` trough the command line interface (see [video guide]()).

In [4]:
fp_json = os.path.join(example_root, 'intermediate', 'MCF10A_demo_GT.json')
fp_pcna = os.path.join(example_root, 'demo', 'MCF10A_demo_PCNA.tif')
fp_bf = os.path.join(example_root, 'demo', 'MCF10A_demo_BF.tif')
table, mask = track_GT_json(fp_json=fp_json, fp_pcna=fp_pcna, fp_bf=fp_bf, displace=120, gap_fill=10,
                            sat=1, gamma=1)
io.imsave(os.path.join(example_root, 'intermediate', 'mask.tif'), mask)

table.to_csv(os.path.join(example_root, 'intermediate', 'raw_object.csv'), index=False)
r = Refiner(track=table, mode='TRH', search_range=10, minM=1, maxBG=1, sample_freq=1 / 5,
            threshold_mt_F=100, threshold_mt_T=20)
ann, track_rfd, mt_dic, imprecise = r.doTrackRefine()
s = Resolver(track_rfd, ann, mt_dic, maxBG=1, minS=1, minM=1, minLineage=10, impreciseExit=imprecise)
out = s.doResolve()
out[0].to_csv(os.path.join(example_root, 'intermediate', 'refined.csv'), index=False)
out[1].to_csv(os.path.join(example_root, 'intermediate', 'phase.csv'), index=False)

Frame 134: 66 trajectories present.


{'sample_freq': 0.2, 'meanDisplace': 3.8771039587878553}
Mean size: 85.14517272839825
Object classification corrected by smoothing: 11
Level 1 mitosis:
Found mitosis track: 12
Level 2 mitosis:
Found mitosis track: 0
High quality tracks subjected to predict relationship: 86
Extracting features...
100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:03<00:00,  5.46it/s]
Finished feature extraction: 153 samples.
Finished prediction.
Register: 22-61
Register: 31-71
Register: 36-73
Register: 38-79
Register: 42-60
Register: 48-63
Register: 56-75
Register: 59-77
Parent-Daughter-Daughter mitosis relations found: 8
Parent-Daughter mitosis relations found: 4
Imprecise tracks involved in prediction: 0
Resolving cell cycle phase...


Phase not resolved yet. Using predicted phase classifications.


---

### 3 Re-resolve the corrected object table

Suppose we have corrected the __`tracked object table`__. Since the correction is on _object level_ rather than _track level_, there will be unresolved cell cycle phases. To resolve again, use `pcnaDeep.resolver.resolve_from_gt`

In [5]:
from pcnaDeep.resolver import resolve_from_gt
help(resolve_from_gt)

Help on function resolve_from_gt in module pcnaDeep.resolver:

resolve_from_gt(track, gt_name='predicted_class', extra_gt=None, G2_trh=None, no_cls_GT=False, minG=1, minS=1, minM=1, minLineage=0)
    Resolve cell cycle phase from the ground truth. Wrapper of `get_rsv_input_gt()`.
    
    Args:
        track (pandas.DataFrame): data frame of each object each row, must have following columns:
            - trackId, frame, parentTrackId, <ground truth classification column>
        gt_name (str): refers to the column in track that corresponds to ground truth classification.
        extra_gt (str): refers to the column in track that has G2 ground truth if `gt_name` does not. See notes below.
        G2_trh (int): intensity threshold for classifying G2 phase (for arrest tracks only).
        no_cls_GT (bool): Set to `true` if no classification ground truth is provided.
            Will resolve based on current classifications.
        minG (int): minimum G phase frame length (default 1).
 

In [6]:
track = pd.read_csv(os.path.join(example_root, 'intermediate', 'refined_corrected.csv'))
out = resolve_from_gt(track, no_cls_GT=False, gt_name='resolved_class', G2_trh=100, minLineage=10)
out[0].to_csv(os.path.join(example_root, 'intermediate', 'refined_corrected_resolved.csv'), index=False)
out[1].to_csv(os.path.join(example_root, 'intermediate', 'refined_corrected_phase.csv'), index=False)

Resolving cell cycle phase...


Using G2 intensity threshold: 100


---

In [2]:
from pyversions import versions
import skimage
versions();

Darwin 21.2.0 64-bit Darwin Kernel Version 21.2.0: Sun Nov 28 20:28:54 PST 2021; root:xnu-8019.61.5~1/RELEASE_X86_64
CPython 3.7.9 (default, Aug 31 2020, 07:22:35) [Clang 10.0.0 ]
Jun 09 2022, 16:53:14

Module           Version
ipython           7.27.0
jupyterlab        3.0.10
logging          0.5.1.2
notebook           6.4.0
numpy             1.18.5
pandas             1.2.4
pyversions         0.0.3
skimage           0.17.2
